In [1]:
import os
import open_clip
import torch
from PIL import Image
import numpy as np
from tqdm import tqdm
import sys

In [2]:
sys.path.append('..')

In [3]:
from config import CLIP_FEATURES_L14_PATH, IMPROVED_KEYFRAME_PATH, CLIP_FEATURES_H14_PATH

In [4]:
from utils.utils import start_from_index
%run ../utils/utils.py

In [5]:
keyframes_dir = "../" + IMPROVED_KEYFRAME_PATH
clip_features_dir = "../" + CLIP_FEATURES_H14_PATH

In [6]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [7]:
# model, _, preprocess = open_clip.create_model_and_transforms('ViT-L-14', device=device, pretrained='datacomp_xl_s13b_b90k')

e:\AICHCM2024\.venv\Lib\site-packages\open_clip\factory.py:129: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path, map_location=map_locat

In [7]:
model, _, preprocess = open_clip.create_model_and_transforms('ViT-H-14-378-quickgelu', device=device, pretrained='dfn5b')

e:\AICHCM2024\.venv\Lib\site-packages\open_clip\factory.py:129: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path, map_location=map_locat

In [8]:
def process_keyframes_in_batch(folder_path, output_path, batch_size):
    video_features = []
    keyframe_paths = sorted([os.path.join(folder_path, img) for img in os.listdir(folder_path)])

    for i in range(0, len(keyframe_paths), batch_size):
        keyframes = []
        batch_paths = keyframe_paths[i:i+batch_size]
        for keyframe_path in batch_paths:
            keyframe = preprocess(Image.open(keyframe_path)).unsqueeze(0)
            keyframes.append(keyframe)

        keyframes = torch.cat(keyframes).to(device)
        
        with torch.no_grad(), torch.amp.autocast("cuda"):
            keyframe_features = model.encode_image(keyframes)
        keyframe_features /= keyframe_features.norm(dim=-1, keepdim=True)

        for idx in range(keyframe_features.shape[0]):
            video_features.append(keyframe_features[idx].detach().cpu().numpy().astype(np.float32).flatten())

    np.save(output_path, video_features)

In [9]:
keyframe_files_to_process = start_from_index(keyframes_dir, 25, "")

In [10]:
len(keyframe_files_to_process)

745

In [11]:
batch_size = 64
for video_folder in tqdm(keyframe_files_to_process):
    video_folder_path = os.path.join(keyframes_dir, video_folder)

    output_path = os.path.join(clip_features_dir, f"{video_folder}.npy")
    process_keyframes_in_batch(video_folder_path, output_path, batch_size)

  0%|          | 0/745 [00:00<?, ?it/s]e:\AICHCM2024\.venv\Lib\site-packages\torch\nn\functional.py:5560: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = scaled_dot_product_attention(q, k, v, attn_mask, dropout_p, is_causal)
100%|██████████| 745/745 [8:14:51<00:00, 39.85s/it]   
